In [40]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw
import re

# EasyOCR reader 객체 생성 (한글 및 영어 지원)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

def process_image(img):
    # 이미지 읽기
    image = cv2.imread(img)
    if image is None:
        raise Exception(f"이미지를 읽을 수 없습니다: {img}")
    
    # 이미지에서 텍스트 추출
    results = reader.readtext(image)
    
    # 추출된 텍스트를 저장할 리스트
    extracted_text = []
    
    # 파란색 상자를 그리기 위한 PIL 이미지로 변환
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    
    # 각 텍스트 주위에 상자를 그리고 텍스트를 저장
    for detection in results:
        bbox = detection[0]
        text = detection[1]
        extracted_text.append(text)
        
        # 상자 좌표
        x_min, y_min = int(bbox[0][0]), int(bbox[0][1])
        x_max, y_max = int(bbox[2][0]), int(bbox[2][1])
        
        # 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
        draw.text((x_min, y_min - 20), text, fill="blue")
    
    # 정보 추출 함수들
    def extract_business_numbers(text_list):
        business_number_pattern = re.compile(r'\b\d{3}-?\d{2}-?\d{5}\b')
        return [match for text in text_list for match in business_number_pattern.findall(text)]
    
    def extract_prices(text_list):
        price_pattern = re.compile(r'결제금액.*?([₩$€]?\s?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)')
        return [match for text in text_list for match in price_pattern.findall(text)]
    
    def extract_store_names(text_list):
        # 오타를 포함한 키워드 패턴
        store_name_pattern = re.compile(r'(상호명|회사명|업체명|가맹점명|가맣점명|상오|[상싱성][호오]|[회훼]사)\s*[:;：]?\s*([^\s)]+(?:\s*\S*?\(?\S*?[점]\)?\s*?\S*)?)')
        return [match[1] for text in text_list for match in store_name_pattern.findall(text)]

    def extract_transaction_date(text_list):
        # 다양한 날짜/시간 형식을 허용하는 정규식
        date_pattern = re.compile(
            r'([거기][래레][일닐][시자]|[결겔]제[일닐]시|거[래레]일|[결겔]제날짜|날짜)\s*[:;：]?\s*(\d{2,4}[-/]\d{2}[-/]\d{2})\s*([0-2]?\d)?[,:]?\s*([0-5]?\d)?[,:]?\s*([0-5]?\d)?')
    
        extracted_dates = []
    
        for text in text_list:
            for match in date_pattern.findall(text):
                # match의 길이에 따라 처리
                if len(match) == 5:  # 모든 정보가 있는 경우
                    hour = match[2] if match[2] else '00'
                    minute = match[3] if match[3] else '00'
                    second = match[4] if match[4] else '00'
                    extracted_dates.append(f"{match[1]} {match[2]}:{minute}:{second}")
                elif len(match) == 4:  # 시간 정보가 없는 경우
                    extracted_dates.append(f"{match[1]} {match[2]}")
                elif len(match) == 3:  # 날짜만 있는 경우
                    extracted_dates.append(f"{match[1]} {match[2]}")
                # 더 많은 조건을 추가하여 처리 가능
    
        return extracted_dates


    def extract_item_prices(text_list):
        item_price_pattern = re.compile(r'(품목|아이템|상품)\s*[:：]?\s*(.*?)\s*가격\s*[:;：]?\s*([₩$€]?\s?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)')
        return [(match[1], match[2]) for text in text_list for match in item_price_pattern.findall(text)]
    
    def extract_total_price(text_list):
        total_price_pattern = re.compile(r'총\s*가격\s*[:;：]?\s*([₩$€]?\s?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)')
        return [match for text in text_list for match in total_price_pattern.findall(text)]
    
    # 정보 추출
    info_dict = {
        "사업자번호": extract_business_numbers(extracted_text),
        "가격": extract_prices(extracted_text),
        "가맹점명": extract_store_names(extracted_text),
        "거래일시": extract_transaction_date(extracted_text),
        "품목별 가격": extract_item_prices(extracted_text),
        "총 가격": extract_total_price(extracted_text)
    }
    
    return info_dict

# 이미지 처리 및 정보 추출
a = process_image('bill4.jpeg')
print(json.dumps(a, ensure_ascii=False, indent=4))


Using CPU. Note: This module is much faster with a GPU.


{
    "사업자번호": [
        "1018126409"
    ],
    "가격": [],
    "가맹점명": [],
    "거래일시": [
        "24/06/24 23:52:04"
    ],
    "품목별 가격": [],
    "총 가격": []
}
